# Imports

In [ ]:
# Basics
import pandas as pd # Data Manipulation
import numpy as np # Linear Algebra

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# SK-learn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import iqr


# OS
import os
from pathlib import Path

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [ ]:
data = {}

In [ ]:
path = 'raw_data'

for i in os.listdir(path):
    subfolder = str(i)
    subfolder = os.path.join(path,subfolder)
    for j in os.listdir(subfolder):
        dict_key = j[:-4]
        if "TeamSpellings" not in j:
            data[dict_key] = pd.read_csv(os.path.join(subfolder,j))
        else:
            data[dict_key] = pd.read_csv(os.path.join(subfolder,j), encoding='cp1252')

### Set Variables

In [ ]:
MNCAATourneyCompactResults = data['MNCAATourneyCompactResults']
MNCAATourneySeeds = data['MNCAATourneySeeds']
MRegularSeasonCompactResults = data['MRegularSeasonCompactResults']
MSeasons = data['MSeasons']
MTeams = data['MTeams']
SampleSubmission2023 = data['SampleSubmission2023']
WNCAATourneyCompactResults = data['WNCAATourneyCompactResults']
WNCAATourneySeeds = data['WNCAATourneySeeds']
WRegularSeasonCompactResults = data['WRegularSeasonCompactResults']
WSeasons = data['WSeasons']
WTeams = data['WTeams']
MNCAATourneyDetailedResults = data['MNCAATourneyDetailedResults']
MRegularSeasonDetailedResults = data['MRegularSeasonDetailedResults']
WNCAATourneyDetailedResults = data['WNCAATourneyDetailedResults']
WRegularSeasonDetailedResults = data['WRegularSeasonDetailedResults']
Cities = data['Cities']
MGameCities = data['MGameCities']
WGameCities = data['WGameCities']
MMasseyOrdinals = data['MMasseyOrdinals']
Conferences = data['Conferences']
MConferenceTourneyGames = data['MConferenceTourneyGames']
MNCAATourneySeedRoundSlots = data['MNCAATourneySeedRoundSlots']
MNCAATourneySlots = data['MNCAATourneySlots']
MSecondaryTourneyCompactResults = data['MSecondaryTourneyCompactResults']
MSecondaryTourneyTeams = data['MSecondaryTourneyTeams']
MTeamCoaches = data['MTeamCoaches']
MTeamConferences = data['MTeamConferences']
MTeamSpellings = data['MTeamSpellings']
WNCAATourneySlots = data['WNCAATourneySlots']
WTeamConferences = data['WTeamConferences']
WTeamSpellings = data['WTeamSpellings']

In [ ]:
all_data_li = [MNCAATourneyCompactResults,
MNCAATourneySeeds,
MRegularSeasonCompactResults,
MSeasons,
MTeams,
SampleSubmission2023,
WNCAATourneyCompactResults,
WNCAATourneySeeds,
WRegularSeasonCompactResults,
WSeasons,
WTeams,
MNCAATourneyDetailedResults,
MRegularSeasonDetailedResults,
WNCAATourneyDetailedResults,
WRegularSeasonDetailedResults,
Cities,
MGameCities,
WGameCities,
MMasseyOrdinals,
Conferences,
MConferenceTourneyGames,
MNCAATourneySeedRoundSlots,
MNCAATourneySlots,
MSecondaryTourneyCompactResults,
MSecondaryTourneyTeams,
MTeamCoaches,
MTeamConferences,
MTeamSpellings,
WNCAATourneySlots,
WTeamConferences,
WTeamSpellings
]

### Seperate by Category

In [ ]:
men = []
women = []
basic = []

In [ ]:
for i,j in enumerate(data):
    if j[0] == 'M':
        men.append(all_data_li[i])
    elif j[0] == 'W':
        women.append(all_data_li[i])
    else:
        basic.append(all_data_li[i])

In [ ]:
print(f"Number of men's datasets: {len(men)}")
print(f"Number of women's datasets: {len(women)}")
print(f"Number of other datasets: {len(basic)}")

Number of men's datasets: 17
Number of women's datasets: 11
Number of other datasets: 3


# Create Training Data

### Combine Women and Men Data

In [ ]:
df_seeds = pd.concat([
    MNCAATourneySeeds,
    WNCAATourneySeeds], ignore_index=True
)
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [ ]:
df_season_results = pd.concat([
    MRegularSeasonCompactResults,
    WRegularSeasonCompactResults,
], ignore_index=True)

df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)
df_season_results['ScoreGap'] = df_season_results['WScore'] - df_season_results['LScore']
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,ScoreGap
0,1985,20,1228,81,1328,64,17
1,1985,25,1106,77,1354,70,7
2,1985,25,1112,63,1223,56,7
3,1985,25,1165,70,1432,54,16
4,1985,25,1192,86,1447,74,12


### Extract Features

In [ ]:
num_win = df_season_results.groupby(['Season', 'WTeamID']).count()
num_win = num_win.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})

In [ ]:
num_loss = df_season_results.groupby(['Season', 'LTeamID']).count()
num_loss = num_loss.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})

In [ ]:
num_win = num_win.sort_values('NumWins',ascending=False)
num_win.head()

,Season,TeamID,NumWins
15202,2015,1246,34
14772,2014,3163,34
13355,2012,3124,34
12016,2010,3163,33
4372,1998,3397,33


In [ ]:
num_loss = num_loss.sort_values('NumLosses',ascending=True)
num_loss.head()

,Season,TeamID,NumLosses
4032,1998,1343,1
17548,2018,3124,1
3472,1996,1403,1
12994,2011,3453,1
4950,1999,3345,1


In [ ]:
gap_win = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()
gap_win = gap_win[['Season', 'WTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapWins", "WTeamID": "TeamID"})
gap_win.sort_values('GapWins')
gap_win.head()

,Season,TeamID,GapWins
0,1985,1102,10.000000
1,1985,1103,7.555556
2,1985,1104,13.190476
3,1985,1106,9.500000
4,1985,1108,13.842105


In [ ]:
gap_loss = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()
gap_loss = gap_loss[['Season', 'LTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapLosses", "LTeamID": "TeamID"})
gap_loss.sort_values('GapLosses')
gap_loss.head()

,Season,TeamID,GapLosses
0,1985,1102,9.947368
1,1985,1103,9.857143
2,1985,1104,4.777778
3,1985,1106,13.285714
4,1985,1108,10.666667


### Merge

In [ ]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

df_features_season = pd.concat([df_features_season_w, df_features_season_l], axis=0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')

df_features_season.fillna(0, inplace=True)  
df_features_season.head()

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses
0,1985,1102,5.0,19.0,10.000000,9.947368
1,1985,1103,9.0,14.0,7.555556,9.857143
2,1985,1104,21.0,9.0,13.190476,4.777778
3,1985,1106,10.0,14.0,9.500000,13.285714
4,1985,1108,19.0,6.0,13.842105,10.666667


In [ ]:

df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLosses'] * df_features_season['GapLosses'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)

df_features_season.drop(['NumWins', 'NumLosses', 'GapWins', 'GapLosses'], axis=1, inplace=True)
df_features_season.head()

,Season,TeamID,WinRatio,GapAvg
0,1985,1102,0.208333,-5.791667
1,1985,1103,0.391304,-3.043478
2,1985,1104,0.700000,7.800000
3,1985,1106,0.416667,-3.791667
4,1985,1108,0.760000,7.960000


In [ ]:

Season	DayNum	WTeamID	WScore	LTeamID	LScore	SeedW	SeedL
0	2016	137	3107	61	3196	59	12	5
1	2016	137	3113	74	3308	52	2	15
2	2016	137	3120	68	3385	57	9	8
3	2016	137	3124	89	3225	59	1	16
4	2016	137	3177	97	3241	67	6	11
5	2016	137	3243	56	3203	51	9	8
6	2016	137	3257	87	3146	60	3	14
7	2016	137	3277	74	3125	60	4	13
8	2016	137	3280	60	3151	50	5	12
9	2016	137	3326	88	3138	69	3	14
10	2016	137	3333	73	3407	31	2	15
11	2016	137	3376	77	3239	41	1	16
12	2016	137	3382	65	3329	54	10	7
13	2016	137	3393	73	3119	56	4	13
14	2016	137	3397	59	3453	53	7	10
15	2016	137	3452	74	3343	65	6	11
16	2016	138	3163	101	3352	49	1	16
17	2016	138	3182	97	3371	76	9	8
18	2016	138	3199	72	3292	55	5	12
19	2016	138	3231	62	3208	58	9	8
20	2016	138	3246	85	3421	31	3	14
21	2016	138	3268	74	3233	58	2	15
22	2016	138	3281	78	3140	69	10	7
23	2016	138	3323	95	3299	61	1	16
24	2016	138	3328	61	3345	45	6	11
25	2016	138	3355	74	3274	71	12	5
26	2016	138	3378	48	3161	45	6	11
27	2016	138	3390	85	3362	58	4	13
28	2016	138	3400	86	3106	42	2	15
29	2016	138	3401	74	3283	65	4	13
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsW',
    'NumLosses': 'NumLossesW',
    'GapWins': 'GapWinsW',
    'GapLosses': 'GapLossesW',
    'WinRatio': 'WinRatioW',
    'GapAvg': 'GapAvgW',
}).drop(columns='TeamID', axis=1)

df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsL',
    'NumLosses': 'NumLossesL',
    'GapWins': 'GapWinsL',
    'GapLosses': 'GapLossesL',
    'WinRatio': 'WinRatioL',
    'GapAvg': 'GapAvgL',
}).drop(columns='TeamID', axis=1)

df.head()

# Preprocessing

### Outliers

In [ ]:
def remove_outliers_tukey(data, alpha=1.5):

    # Select only the numerical columns
    num_cols = data.select_dtypes(include=[np.number]).columns
    data_num = data[num_cols]
    
    # Compute the first and third quartiles
    q1, q3 = np.percentile(data_num, [25, 75])
    
    # Compute the interquartile range (IQR)
    iqr_val = iqr(data_num)
    
    # Compute the range outside of which data points are considered outliers
    outlier_range = (q1 - alpha * iqr_val, q3 + alpha * iqr_val)
    
    # Identify the outliers and remove them
    outliers = (data_num < outlier_range[0]) | (data_num > outlier_range[1])
    data_num_no_outliers = data_num[~outliers]
    
    # Merge the numerical columns back into the original data frame
    data_no_outliers = pd.concat([data_num_no_outliers, data.select_dtypes(exclude=[np.number])], axis=1)
    return data_no_outliers

data_with_removed_outliers = remove_outliers_tukey(train_df)
data_with_removed_outliers.shape